In [12]:
# coding: utf-8

get_ipython().magic('matplotlib')

import sys
import os
import argparse
import glob
import time
import re
import numpy as np
import scipy
import pandas as pd
import pims
from PIL import Image
from skimage import io
from skimage import color as col
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import Formatter, Locator, NullLocator, FixedLocator, NullFormatter



#parser = argparse.ArgumentParser();
#parser.add_argument('-i', '--image', help='a directory that contains all the images for processing');
#parser.add_argument('-d', '--data', help='a directory that contains tracking data files for the images to process');

#arg = parser.parse_args();

#image_dir = arg.image;
#data_dir = arg.data;
image_dir = '/Users/Jeff/Desktop/Batch09/';
data_dir = '/Users/Jeff/Desktop/Batch09/120_spots/'
os.chdir(data_dir);
C1_data_file = glob.glob('C1*.txt');
C2_data_file = glob.glob('C2*.txt');
C1_data = pd.read_table(C1_data_file[0]);
C2_data = pd.read_table(C2_data_file[0]);


num_frames = 100;
frame_number = 1;
os.chdir(image_dir);
image_list = glob.glob('*.tif');
for i in image_list:
    cur_image = i;
    m = re.search('(Batch\d+_.*_XY\d+).*.tif', cur_image);
    match = m.group(1);
    
    im = io.imread(cur_image)
    
    x = C1_data[C1_data.C1_Image.str.contains(match)];
    singlets_c1 = x.C1_Image.unique();
    
    x2 = C2_data[C2_data.C2_Image.str.contains(match)];
    singlets_c2 = x2.C2_Image.unique();
    
    fig = plt.figure(figsize=(5,5), frameon=False);
    ax = fig.add_axes([0,0,1,1], xlim=(0,512), ylim=(512,0), aspect='equal', frameon=False);
    ax.set_axis_off();
    plt.margins(0,0)
    ax.set_frame_on(False);
    ax.set_xlim([0, 512]);
    ax.set_ylim([512, 0]);
    
    
    
    num_singlets = singlets_c1.size;
    x_location = np.full(num_singlets, np.nan);
    x2_location = np.full(num_singlets, np.nan);
    y_location = np.full(num_singlets, np.nan);
    y2_location = np.full(num_singlets, np.nan);
    
    im_now = im[frame_number, :, :, :];
    
    x_line = np.full((num_singlets, num_frames), np.nan);
    y_line = np.full((num_singlets, num_frames), np.nan);
    x2_line = np.full((num_singlets, num_frames), np.nan);
    y2_line = np.full((num_singlets, num_frames), np.nan);
    
    
    implt = plt.imshow(im_now);
    roi_1 = patches.Rectangle((80,98), 42, 42, fill=False, linewidth=1.5, color='yellow');
    ax.add_patch(roi_1);
    roi_2 = patches.Rectangle((406,278), 42, 42, fill=False, linewidth=1.5, color='yellow');
    ax.add_patch(roi_2);
    #roi_3 = patches.Rectangle((262,82), 42, 42, fill=False, linewidth=1.5, color='yellow');
    #ax.add_patch(roi_3);
    #roi_4 = patches.Rectangle((336,23), 53, 53, fill=False, linewidth=1.5, color='yellow');
    #ax.add_patch(roi_4);

    
    for j in range(0,num_singlets):
        
        if not x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_X.values.size == 0:
            x_location[j] = x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_X.values;
            y_location[j] = x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_Y.values;
        else:
            x_location[j] = np.nan;
            y_location[j] = np.nan;
        if not x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_X.values.size == 0:
            x2_location[j] = x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_X.values;
            y2_location[j] = x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_Y.values;
        else:
            x2_location[j] = np.nan;
            y2_location[j] = np.nan;

    if not x_location.size == 0:
        scat = ax.scatter([x_location], [y_location], marker='+', c='0.75', s=60, linewidth=1.5);
    if not x2_location.size == 0:
        scat2 = ax.scatter([x2_location], [y2_location], marker='+', c='0.3', s=60, linewidth=1.5);

    lines_c1 = [];
    lines_c2 = [];

    for j in range(0,num_singlets):
        lobj = ax.plot([], [], c='0.9', linewidth=0.75)[0];
        lines_c1.append(lobj);
        lobj = ax.plot([], [], c='0.5', linewidth=0.75)[0];
        lines_c2.append(lobj);


    def update (frame_number):
        im_now = im[frame_number, :, :, :];
        implt.set_data(im_now);
        for j in range(0,num_singlets):
            if not x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_X.values.size == 0:
                x_location[j] = x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_X.values;
                y_location[j] = x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_Y.values;
            else:
                x_location[j] = np.nan;
                y_location[j] = np.nan;
            if not x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_X.values.size == 0:
                x2_location[j] = x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_X.values;
                y2_location[j] = x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_Y.values;
            else:
                x2_location[j] = np.nan;
                y2_location[j] = np.nan;

        if not x_location.size == 0:
            data = np.stack(([x_location], [y_location]), axis=-1)
            scat.set_offsets(data);
            x_line[:, frame_number - 1] = x_location;
            y_line[:, frame_number - 1] = y_location;

        if frame_number > 1:
            for lnum, line in enumerate(lines_c1):
                line.set_data(x_line[lnum, :frame_number], y_line[lnum, :frame_number]);
        
            if not x2_location.size == 0:
                data2 = np.stack(([x2_location], [y2_location]), axis=-1)
                scat2.set_offsets(data2);
                x2_line[:, frame_number - 1] = x2_location;
                y2_line[:, frame_number - 1] = y2_location;

        if frame_number > 1:
            for lnum, line in enumerate(lines_c2):
                line.set_data(x2_line[lnum, :frame_number], y2_line[lnum, :frame_number]);
        
        return

m = re.search('(Batch\d+_.*_XY\d+.*).tif', cur_image);
match = m.group(1);
animation = FuncAnimation(fig, update, frames=100, interval=80, repeat=True);
animation.save(match + '.mp4', writer='ffmpeg', dpi=300, bitrate=12000, fps=14);
plt.show();



Using matplotlib backend: MacOSX


In [ ]:
x_location

In [ ]:
num_singlets

In [ ]:
x.C1_Image.unique()

In [ ]:
x[(x.C1_FRAME == 0) & (x.C1_Image == singlets_c1[1])].C1_Position_X.values.size == 0

In [ ]:
x[(x.C1_FRAME == 0)].C1_Position_X.values

In [ ]:
    frame_number = 0
    for j in range(0,num_singlets):    
        
        if not x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_X.values.size == 0:
            x_location[j] = x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_X.values;   
            y_location[j] = x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1[j])].C1_Position_Y.values;   
        else:
            x_location[j] = np.nan;
            y_location[j] = np.nan;
        if not x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_X.values.size == 0:
            x2_location[j] = x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_X.values;   
            y2_location[j] = x2[(x2.C2_FRAME == frame_number) & (x2.C2_Image == singlets_c1[j])].C2_Position_Y.values;   
        else:
            x2_location[j] = np.nan;
            y2_location[j] = np.nan;


In [ ]:
x_location

In [ ]:
scat = ax.scatter([x_location[0]], [y_location[0]], marker='+', c='r', s=120);



In [ ]:
import matplotlib;
matplotlib.rcParams['animation.codec']

In [ ]:
tup = (5, 5);
fig = plt.figure(figsize=tup, frameon=False, tight_layout=True);
ax=fig.gca();
ax.set_xlim([0, 512]);
ax.set_ylim([512, 0]);
ax.set_xticks([]);
ax.set_yticks([]);
ax.set_frame_on(False);
implt = plt.imshow(im_now);

In [ ]:
ax=fig.gca();

In [ ]:
ax.set_frame_on(False);

In [ ]:
fig.tight_layout();

In [ ]:
ax.set_xlim([0, 512]);
ax.set_ylim([512, 0]);
ax.set_xticks([]);
ax.set_yticks([]);

In [ ]:
implt = plt.imshow(im_now);

In [ ]:
ax.set_xlim([0, 512]);
ax.set_ylim([512, 0]);
ax.set_xticks([]);
ax.set_yticks([]);

In [ ]:
implt = plt.imshow(im_now);

In [ ]:
fig = plt.figure(frameon=False);

In [ ]:
ax=fig.gca();


In [ ]:
implt.axes.axis('tight')

In [ ]:
fig = plt.figure(figsize=(5,5), frameon=False);

In [ ]:
# coding: utf-8

get_ipython().magic('matplotlib')

import sys
import os
import argparse
import glob
import time
import re
import numpy as np
import scipy
import pandas as pd
import pims
from PIL import Image
from skimage import io
from skimage import color as col
import matplotlib.pyplot as plt
import matplotlib.lines as lines
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import Formatter, Locator, NullLocator, FixedLocator, NullFormatter



In [ ]:
fig = plt.figure(figsize=(5,5), frameon=False);
ax = fig.add_axes([0,0,1,1], xlim=(0,512), ylim=(512,0), aspect='equal', frameon=False);
implt = plt.imshow(im_now);
rect = patches.Rectangle((20,20), 50, 50, fill=False, linewidth=1.5, color='w', linestyle='dashed');
ax.add_patch(rect);

In [ ]:
im.shape()

In [ ]:
im.shape[0]

In [ ]:
singlets_c1

In [ ]:
singlets_c1

In [ ]:
m = re.search('(Batch\d+_.*_XY\d+.*_singlet\d+).*.tif', cur_image);
match = m.group(1);

In [ ]:
match

In [ ]:
    singlet_c1 = x.C1_Image.unique()[0];



In [ ]:
singlet_c1

In [ ]:
x.C1_Image == singlet_c1

In [ ]:
if not x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1)].C1_Position_X.values.size == 0:
    singlet_c1


In [ ]:
x.C1_FRAME == frame_number

In [ ]:
x[(x.C1_FRAME == frame_number) & (x.C1_Image == singlets_c1)]

In [ ]:
(x.C1_Image == singlets_c1)

In [19]:
fig = plt.figure(figsize=(5,5), frameon=False);
ax = fig.add_axes([0,0,1,1], xlim=(0,512), ylim=(512,0), aspect='equal', frameon=False);
ax.set_axis_off();
plt.margins(0,0)
ax.set_frame_on(False);
ax.set_xlim([0, 512]);
ax.set_ylim([512, 0]);
im_now = im[1, :, :, :];
plt.imshow(im_now);
roi_1 = patches.Rectangle((80,98), 42, 42, fill=False, linewidth=1.5, color='yellow', );
ax.add_patch(roi_1);
roi_2 = patches.Rectangle((406,278), 42, 42, fill=False, linewidth=1.5, color='yellow');
ax.add_patch(roi_2);
roi_3 = patches.Rectangle((262,82), 42, 42, fill=False, linewidth=1.5, color='yellow');
ax.add_patch(roi_3);
roi_4 = patches.Rectangle((336,23), 53, 53, fill=False, linewidth=1.5, color='yellow');
ax.add_patch(roi_4);
fig.savefig('Batch09_08_XY02_frame1.png');